<a href="https://colab.research.google.com/github/ShauryaDusht/machine-learning-basics/blob/main/LAB3_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 3 - Shaurya Bisht UI22EC67**



## (a) To collect tweets that may incorporate owner, date of post, number of re-tweet, no of followers, and other associated information from Twitter and store it into a .csv file.


In [1]:
!pip install ntscraper

In [4]:
from ntscraper import Nitter
import time
data = None
def scrape_twitter(username, retries=3):
    for attempt in range(retries):
        try:
            # Create a new scraper instance
            scraper = Nitter()

            # Explicitly get available instances
            print("Available instances:", scraper.instances)

            # Get tweets
            tweets_data = scraper.get_tweets(
                username,
                mode='user',
                number=10
            )

            if tweets_data and 'tweets' in tweets_data:
                data = {
                    'link': [],
                    'text': [],
                    'user': [],
                    'likes': [],
                    'retweets': [],
                    'comments': []
                }

                for tweet in tweets_data['tweets']:
                    try:
                        data['link'].append(tweet['link'])
                        data['text'].append(tweet['text'])
                        data['user'].append(tweet['user']['name'])
                        data['likes'].append(tweet['stats']['likes'])
                        data['retweets'].append(tweet['stats']['retweets'])
                        data['comments'].append(tweet['stats']['comments'])
                    except KeyError as e:
                        print(f"Missing key in tweet data: {e}")
                        continue

                return data

        except Exception as e:
            print(f"Attempt {attempt + 1} failed with error: {e}")
            if attempt < retries - 1:
                print("Waiting 5 seconds before retry...")
                time.sleep(5)
            else:
                print("All retry attempts failed")
                return None

# Run the scraper
result = scrape_twitter("narendramode")
if result:
    print("\nSuccessfully scraped tweets:")
    print(result)

Testing instances: 100%|██████████| 6/6 [00:03<00:00,  1.85it/s]
INFO:root:No instance specified, using random instance https://lightbrd.com


Available instances: ['https://xcancel.com', 'https://nitter.privacydev.net', 'https://nitter.poast.org', 'https://lightbrd.com', 'https://nitter.space', 'https://nitter.privacyredirect.com']
Attempt 1 failed with error: list index out of range
Waiting 5 seconds before retry...


Testing instances: 100%|██████████| 6/6 [00:03<00:00,  1.80it/s]
INFO:root:No instance specified, using random instance https://nitter.privacyredirect.com


Available instances: ['https://xcancel.com', 'https://nitter.privacydev.net', 'https://nitter.poast.org', 'https://lightbrd.com', 'https://nitter.space', 'https://nitter.privacyredirect.com']


INFO:root:Current stats for narendramode: 3 tweets, 0 threads...



Successfully scraped tweets:
{'link': ['https://twitter.com/narendramode/status/276297979101601792#m', 'https://twitter.com/narendramode/status/276297276333371392#m', 'https://twitter.com/narendramode/status/276292296549482496#m'], 'text': ['नरेन्द्र मोदी की सच्चाई  narindramodi.blogspot.com/?s…', 'मोदी की जुबानी , गुजरात की कहानी : The truth behind Modi’s claims on development modikigujarat.blogspot.com/2…', 'About 89% men and 98% of women workers in Gujarat get low wages, have poor working conditions & low social protection.'], 'user': ['narendra mode', 'narendra mode', 'narendra mode'], 'likes': [24, 18, 18], 'retweets': [2, 0, 0], 'comments': [5, 2, 3]}


In [6]:
import pandas as pd
df = pd.DataFrame(result)
df.to_csv('tweets.csv', index=False)
print("Tweets saved to tweets.csv")

Tweets saved to tweets.csv


In [7]:
df.head()

,link,text,user,likes,retweets,comments
0,https://twitter.com/narendramode/status/276297...,नरेन्द्र मोदी की सच्चाई narindramodi.blogspot...,narendra mode,24,2,5
1,https://twitter.com/narendramode/status/276297...,"मोदी की जुबानी , गुजरात की कहानी : The truth b...",narendra mode,18,0,2
2,https://twitter.com/narendramode/status/276292...,About 89% men and 98% of women workers in Guja...,narendra mode,18,0,3


## (b) To scrap users reviews from any E-commerce portal (Ex- Amazon, Flipkart) and store it into a csv file that may incorporate date of post, number of likes/dislikes, and other associated fields.

In [8]:
from bs4 import BeautifulSoup
import requests
import time
import random
import csv

def fetch_page(URL):
    """Fetches the HTML content of the given Amazon URL."""
    HEADERS = {
        'User-Agent': random.choice([
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0.0',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
        ]),
        'Accept-Language': 'en-US, en;q=0.5',
        'Referer': 'https://www.google.com/',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Connection': 'keep-alive'
    }

    # Adding a delay to prevent detection
    time.sleep(random.uniform(2, 5))

    try:
        response = requests.get(URL, headers=HEADERS, timeout=10)

        # Check for successful response
        if response.status_code == 503:
            print("⚠ 503 Error: Amazon might be blocking the request.")
            return None
        elif response.status_code != 200:
            print(f"❌ Request failed with status code: {response.status_code}")
            return None

        print("✅ Page fetched successfully!")
        return response.content

    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return None

def extract_data(html_content):
    """Extracts product title and review summary from the HTML content."""
    soup = BeautifulSoup(html_content, "lxml")

    # Extract Product Title
    title_element = soup.find("span", {"id": "productTitle"})
    product_title = title_element.get_text(strip=True) if title_element else "NA"

    # Person name
    profile_div = soup.find("div", class_="a-profile-content")
    profile_name = profile_div.find("span", class_="a-profile-name").get_text(strip=True) if profile_div else "NA"

    # Rating
    rating_element = soup.find("i", {"data-hook": "review-star-rating"})
    rating = rating_element.find("span", class_="a-icon-alt").get_text(strip=True) if rating_element else "NA"

    # Review date
    review_date_element = soup.find("span", {"data-hook": "review-date"})
    review_date = review_date_element.get_text(strip=True) if review_date_element else "NA"

    # Extract Review Summary
    review_element = soup.find("span", {"data-hook": "review-body"})
    review_summary = review_element.get_text(strip=True) if review_element else "NA"


    # ✅ Fixed print statements
    print(f"Product Title: {product_title}")
    print(f"Reviewer Name: {profile_name}")
    print(f"Review Date: {review_date}")
    print(f"Review Rating: {rating}")
    print(f"Review Summary: {review_summary}")

    return product_title, profile_name, review_date, rating, review_summary

def save_to_csv(data, filename="amazon_data1.csv"):
    """Saves extracted data to a CSV file."""
    with open(filename, "a", newline='', encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(data)

def main(URL):
    """Main function to scrape Amazon product details."""
    html_content = fetch_page(URL)
    if html_content:
        product_title, profile_name, review_date, rating, review_summary = extract_data(html_content)
        save_to_csv([product_title, profile_name, review_date, rating, review_summary])

if __name__ == "__main__":
    # URL of the Amazon product page
    URL = "https://www.amazon.in/CB-COLEBROOK-Regular-Spread-Collar-XX-Large/dp/B0CGLN88TP/ref=asc_df_B0CGLN88TP/?tag=googleshopdes-21&linkCode=df0&hvadid=710086631690&hvpos=&hvnetw=g&hvrand=18257018832971875730&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9062189&hvtargid=pla-2291007525597&psc=1&mcid=9cbf3f84df2c3a4c92a55c777841505a&gad_source=1"
    main(URL)


✅ Page fetched successfully!
Product Title: CB-COLEBROOK Men's Regular Fit Solid Soft Touch Cotton Casual Shirt with Pocket Design with Spread Collar & Full Sleeves
Reviewer Name: CB-COLEBROOK Men's Regular Fit Solid Soft Touch Cotton Casual Shirt with Pocket Design with Spread
Review Date: Reviewed in India on 23 November 2024
Review Rating: 5.0 out of 5 stars
Review Summary: This shirt is an excellent value for money! It looks exactly like the picture shown, with great attention to detail and quality. The fabric is comfortable and breathable, making it perfect for daily wear. The fit is true to size, and the color and design match perfectly as advertised. I’m really pleased with my purchase and would definitely recommend it to anyone looking for a stylish and affordable shirt.Read more


In [9]:
df = pd.read_csv('amazon_data1.csv')
df.head()

,CB-COLEBROOK Men's Regular Fit Solid Soft Touch Cotton Casual Shirt with Pocket Design with Spread Collar & Full Sleeves,CB-COLEBROOK Men's Regular Fit Solid Soft Touch Cotton Casual Shirt with Pocket Design with Spread,Reviewed in India on 23 November 2024,5.0 out of 5 stars,"This shirt is an excellent value for money! It looks exactly like the picture shown, with great attention to detail and quality. The fabric is comfortable and breathable, making it perfect for daily wear. The fit is true to size, and the color and design match perfectly as advertised. I’m really pleased with my purchase and would definitely recommend it to anyone looking for a stylish and affordable shirt.Read more"
